## Import Stuff

In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu
!pip install -q langchain
!pip install -U langchain-community
!pip install -q streamlit
!pip install pypdf > /dev/null
!pip install unstructured > /dev/null
!pip install jq > /dev/null

## RAG

In [ ]:
%%writefile app.py
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader, CSVLoader, UnstructuredMarkdownLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import os

# Cache the document loading and processing
@st.cache_data
def load_and_process_documents(file_path):
    documents = []
    for file in os.listdir(file_path):
      if file=="logs.txt":
        continue
      if file.endswith('.txt'):
        text_file_path = os.path.join(file_path, file)
        loader = TextLoader(text_file_path)
        documents.extend(loader.load())
      elif file.endswith('.pdf'):
        pdf_file_path = os.path.join(file_path, file)
        loader = PyPDFLoader(pdf_file_path)
        documents.extend(loader.load())
      elif file.endswith('.csv'):
        csv_file_path = os.path.join(file_path, file)
        loader = CSVLoader(csv_file_path)
        documents.extend(loader.load())
      elif file.endswith('.md'):
        md_file_path = os.path.join(file_path, file)
        loader = UnstructuredMarkdownLoader(md_file_path)
        documents.extend(loader.load())
    # loader = TextLoader(file_path)
    # docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
    chunked_docs = splitter.split_documents(documents)
    return chunked_docs

# Cache the model loading
@st.cache_resource
def load_model_and_tokenizer(model_name, bnb_config):
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer


chunked_docs = load_and_process_documents("/content/")

embedding_model = "BAAI/bge-base-en-v1.5"
db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name=embedding_model))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

model_name = "HuggingFaceH4/zephyr-7b-beta"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
model, tokenizer = load_model_and_tokenizer(model_name, bnb_config)

prompt_template = """
Always start your response with "Hey Himanshu!"
Answer the question based on your knowledge. Use the following context to help:
{context}
</s>

{question}
</s>
"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def run_model(inputs):
    context = inputs['context']
    question = inputs['question']
    formatted_input = prompt.format(context=context, question=question)
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=512, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
def run_rag_chain(question):
    context_docs = retriever.get_relevant_documents(question)
    context = " ".join([doc.page_content for doc in context_docs])
    response = run_model({"context": context, "question": question})
    return response

st.title("RAG-based Question Answering")

question = st.text_input("Enter your question:")
if question:
    response = run_rag_chain(question)
    st.write(response)


## Run using localtunnel

In [ ]:
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com